# Réaliser quelques traitements avec Python

Dans ce notebook, on explore différents "outils" permettant de faire des traitements simples comme "pseudonymiser" les entretiens, compter les occurrences d'un mot ou faire ressortir les termes spécifiques d'un entretien.

Certains de ces traitements peuvent être réalisés avec les modules de base de Python, d'autres font appel à des modules spécialisés dans le traitement automatique des langues comme les modules `Spacy` ou `Scikit Learn`.

In [62]:
!pip install pypandoc


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [204]:
### import re # Module pour utiliser les "regex", i.e. les expressions régulières

import pandas as pd # module pour manipuler des "dataframes". 
import csv # Outil pour lire et écrire des fichiers csv
import numpy as nmp

import pypandoc

import fileinput


## Chargement des données

On ouvre les fichiers csv contenant les retranscriptions avec `pandas`. Pour cela nous avons besoin de connaitre l'adress (ou chemin) du fichier et d'utiliser la fonction pd.read_csv(adresse du fichier, sep =","). L'argument "sep" correspond au séparateur utiliser pour marquer les colonnes.

In [64]:
gb_transcript = "../records/grazia_borrini_07-06-18.csv" # Adresse du fichier csv
gb = pd.read_csv(gb_transcript, sep=",")



**Exercice**

Ouvrez le fichier "Marie_Roue_29-03-18.csv" avec pandas. Vous utiliserez "mr" (pour Marie Roué) pour nommer l'objet contenant le dataframe (c'est-à-dire les données du fichier csv).

In [65]:
mr_transcript = "../records/Marie_Roue_29-03-18.csv"
mr = pd.read_csv(mr_transcript, sep=",")

# Regrouper les segments en blocs

Whisper découpe l'entretient en segment de plus ou moins 10 secondes. Pour lire l'entretien, il peut toutefois être plus facile de regrouper les segments liés à une même question par exemple. Le regroupement facilitera également le calcul du "TF-IDF" que l'on verra plus loin.


In [66]:
def get_bloc(data, column, new_dataframe = False) : #fonction pour regrouper les segments d'un même bloc (prise de parole d'un locuteur)
    """Groupe les segments d'une retranscription whisper en bloc.

    ------
    data : le dataframe contenant la retranscription. Le dataframe doit avoir à minima une colonne "segment"
    column : la colonne à partir de laquelle  le regroupement est effectué.
    new_dataframe : Si faux (argument par défaut), la fonction crée une nouvelle colonne au dataframe (dans lequel une ligne est égal à un segment). Si vrai, la fonction retourne un nouveau dataframe dans lequel une ligne est égal un bloc
    """
    
    dict_bloc_id = {}
    for n, speaker in enumerate(data[column]):
        if n == 0:
            bloc_id = n
        else :
            if speaker == data[column].iloc[n-1]:
                bloc_id = int(dict_bloc_id[n-1].replace("bloc_",""))
            else:
                bloc_id = int(dict_bloc_id[n-1].replace("bloc_",""))+1
        dict_bloc_id[n] = "bloc_"+str(bloc_id)
    data["bloc_id"] = data.index.map(dict_bloc_id.get)
    data["bloc_text"] = data.groupby(['bloc_id'])['text'].transform(lambda x : ' '.join(x))
    data["start_bloc"] = data.groupby(['bloc_id'])['start'].transform("min")
    data["end_bloc"] = data.groupby(['bloc_id'])['end'].transform("min")
    data["start_bloc_ms"] = data.groupby(['bloc_id'])['start_in_ms'].transform("min")
    data["end_bloc_ms"] = data.groupby(['bloc_id'])['end_in_ms'].transform("min")
    if new_dataframe==True:
        data = data.drop(columns=["text", "id","start", "end", "start_in_ms", "end_in_ms"]).drop_duplicates()
    return dict_bloc_id, data


def write_bloc_in_list(data, column_of_text = "text", segment_tag=False, segment_id=None):
    """
    Ecrit les blocs obtenu avec la fonction "get_bloc" dans une liste.

    ------
    data: le dataframe contenant la retranscription avec les segments regroupés par bloc.
    column_of_text : le nom de la colonne contenant le text.
    segment_tag : Boolean (Faux par défaut). Si vrai et que le dataframe contient une colonne segment, la fonction ajout un tag sous la forme "<seg_id>segment</seg_id>" pour marquer le début et la fin de chaque segment d'un même bloc.
    segment_id : non de la colonne contenant les idenfitifiants des segments

    Retourne une liste des blocs (utile pour le calcul du tf-idf ensuite)
    """
    
    list_bloc_text = []
    #dict_speaker = {}
    for n, b in enumerate(data.bloc_id.unique()):
        data_tmp = data.loc[data.bloc_id == b]
        if segment_tag == True:
            new_text = ""
            try:
                for m, seg in enumerate(data_tmp[segment_id]):
                    new_text = new_text + "<" + seg + ">" + data_tmp[column_of_text].iloc[m]  + "</" + seg + ">" #décommenter la ligne ci-dessous si on souhaite mettre des balises indiquant le début et la fin des segments que l'on concatène
            except:
                raise
        else:
            new_text = data_tmp[column_of_text].iloc[0] 
        list_bloc_text.append({"bloc_id":b, "speaker": data_tmp.speaker.unique()[0], "bloc_text" : new_text, "start_bloc": data_tmp.start_bloc.iloc[0], "end_bloc": data_tmp.end_bloc.iloc[0]})
    return list_bloc_text
    

In [67]:
dict_bloc_id, gb1 = get_bloc(data=gb, column="speaker", new_dataframe=False)
gb1

,id,start,end,speaker,text,start_in_ms,end_in_ms,bloc_id,bloc_text,start_bloc,end_bloc,start_bloc_ms,end_bloc_ms
0,seg_1,"00:00:01,820","00:00:09,259",Speaker 1,Je suis arrivé en post-doctorat au Muséum Nati...,1.820,9.259,bloc_0,Je suis arrivé en post-doctorat au Muséum Nati...,"00:00:01,820","00:00:09,259",1.820,9.259
1,seg_2,"00:00:09,299","00:00:19,930",Speaker 1,C'est un projet porté par à la fois Stéphanie ...,9.299,19.930,bloc_0,Je suis arrivé en post-doctorat au Muséum Nati...,"00:00:01,820","00:00:09,259",1.820,9.259
2,seg_3,"00:00:20,441","00:00:22,143",Speaker 0,"Oh, Citizen Science, ok.",20.441,22.143,bloc_1,"Oh, Citizen Science, ok.","00:00:20,441","00:00:22,143",20.441,22.143
3,seg_4,"00:00:22,183","00:00:32,453",Speaker 1,Avec l'idée de faire participer le public à la...,22.183,32.453,bloc_2,Avec l'idée de faire participer le public à la...,"00:00:22,183","00:00:32,453",22.183,32.453
4,seg_5,"00:00:32,493","00:00:35,517",Speaker 0,"Peut-être direction de recherche aussi, n'est-...",32.493,35.517,bloc_3,"Peut-être direction de recherche aussi, n'est-...","00:00:32,493","00:00:35,517",32.493,35.517
...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,seg_930,"02:13:20,896","02:13:24,978",Speaker 0,"Bon, il y a plus de responsabilités.",8000.896,8004.978,bloc_109,"La France est particulière, ce sens, n'est-ce ...","02:11:45,664","02:11:48,406",7905.664,7908.406
930,seg_931,"02:13:25,018","02:13:48,798",Speaker 0,"C'est un peu... Évidemment, il y a plus de res...",8005.018,8028.798,bloc_109,"La France est particulière, ce sens, n'est-ce ...","02:11:45,664","02:11:48,406",7905.664,7908.406
931,seg_932,"02:13:48,818","02:13:49,499",Speaker 0,Ça fiche de tout.,8028.818,8029.499,bloc_109,"La France est particulière, ce sens, n'est-ce ...","02:11:45,664","02:11:48,406",7905.664,7908.406
932,seg_933,"02:13:59,889","02:14:03,773",Speaker 1,"Bon, on va essayer.",8039.889,8043.773,bloc_110,"Bon, on va essayer. Merci en tout cas pour...","02:13:59,889","02:14:03,773",8039.889,8043.773


In [68]:
write_bloc_in_list(data=gb1, column_of_text = "bloc_text", segment_tag=False, segment_id="id")


[{'bloc_id': 'bloc_0',
  'speaker': 'Speaker 1',
  'bloc_text': "Je suis arrivé en post-doctorat au Muséum National d'Historie Naturelle l'année dernière, en septembre. C'est un projet porté par à la fois Stéphanie Duvaille et les écologues qui ont mis au point ce qu'on appelle les sciences participatives ou en anglais les citizen science.",
  'start_bloc': '00:00:01,820',
  'end_bloc': '00:00:09,259'},
 {'bloc_id': 'bloc_1',
  'speaker': 'Speaker 0',
  'bloc_text': 'Oh, Citizen Science, ok.',
  'start_bloc': '00:00:20,441',
  'end_bloc': '00:00:22,143'},
 {'bloc_id': 'bloc_2',
  'speaker': 'Speaker 1',
  'bloc_text': "Avec l'idée de faire participer le public à la récolte de données, etc.",
  'start_bloc': '00:00:22,183',
  'end_bloc': '00:00:32,453'},
 {'bloc_id': 'bloc_3',
  'speaker': 'Speaker 0',
  'bloc_text': "Peut-être direction de recherche aussi, n'est-ce pas?",
  'start_bloc': '00:00:32,493',
  'end_bloc': '00:00:35,517'},
 {'bloc_id': 'bloc_4',
  'speaker': 'Speaker 1',
  '

In [69]:
dict_bloc_id, gb1 = get_bloc(data=gb, column="speaker", new_dataframe=False)
list_bloc_text = write_bloc_in_list(data=gb1, column_of_text = "bloc_text", segment_tag=True, segment_id="id")
with open("../records/2026-02-05_grazia_borrini_in_block.txt" , 'w', newline='') as txt:
    for row in list_bloc_text:
        start = row["start_bloc"]
        end = row["end_bloc"]
        speaker = row["speaker"]
        bloc_text = row["bloc_text"]
        txt.write(f"{row["bloc_id"]}\n")
        txt.write(f"{start} --> {end}\n")
        txt.write(f"{speaker} : {bloc_text}\n\n")
            

        

In [70]:
list_of_dataframe = [gb, mr]
list_of_path = [gb_transcript, mr_transcript]
for n, df in enumerate([gb, mr]):
    dict_bloc_id, df1 = get_bloc(data=df, column="speaker", new_dataframe=False)
    list_bloc_text = write_bloc_in_list(data=df1, column_of_text = "bloc_text", segment_tag=False, segment_id=None)
    with open(f"{list_of_path[n]}_in_bloc.txt" , 'w', newline='') as txt:
        for row in list_bloc_text:
            start = row["start_bloc"]
            end = row["end_bloc"]
            speaker = row["speaker"]
            bloc_text = row["bloc_text"]
            txt.write(f"{row["bloc_id"]}\n")
            txt.write(f"{start} --> {end}\n")
            txt.write(f"{speaker} : {bloc_text}\n\n")
    

# Pseudonomyser et anonymiser les locuteurs

Quand on travaille avec des entretiens, il est souvent nécessaire de les pseudonomyser. On peut pour cela utiliser l'outil "find and replace" de son loogiciel de traitement de texte. On peut également le faire avec python, notamment lorsqu'on a de nombreux noms de personnes ou de lieux à remplacer.

C'est ce que propose de faire la fonction ci-dessous. 

In [71]:
def pseudonymiser(original_name, new_name, file, column = None, replace_file=False):
    """
    Pseudonymise un fichier
    ---------------

    original_name : 'string'.
    new_name : 'string'. The pseudonym
    file : path of file to pseudonymise. It could be a csv, srt or txt file.
    column : only if "file" is a csv. Name of the column to pseudonymise
    replace_file: boolean. If false, it create a new file. Else it erases the given file.
    """
    
    if file.split(".")[-1] == "csv":
        with open(file, "r") as csvf:
            reader2 = csv.DictReader(csvf)
            old_row = []
            line_to_override={}
            for n, r in enumerate(reader2):
                old_row.append(r)
                pseudo = r[column].replace(original_name, new_name)
                r[f"pseudo_{column}"] = pseudo
                line_to_override[n] = r
        if replace_file == False:
            new_file = file.replace(".csv", "pseudonymise.csv")
            with open(new_file, 'w', newline='') as csvfile:
                fieldnames = [key for key, value in line_to_override[0].items()]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for line, row in enumerate(old_row):
                    data = line_to_override.get(line, row)
                    #print(data)
                    writer.writerow(data)
            return new_file
        else:
            with open(file, 'w', newline='') as csvfile:
                fieldnames = [key for key, value in line_to_override[0].items()]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for line, row in enumerate(old_row):
                    data = line_to_override.get(line, row)
                    #print(data)
                    writer.writerow(data)
            return file
    else:
        if replace_file == False:
            find_ext = file.split(".")[-1]
            new_file = file.replace(f".{find_ext}", f"pseudonymise.{find_ext}")
            with open(file, 'r', encoding='utf-8-sig') as txt:
                with open(new_file, 'w', encoding='utf-8') as txt2:
                    for line in txt:
                        new_line = re.sub(original_name, new_name, line)
                        txt2.write(new_line)
            return new_file
        else:
            for line in fileinput.input(file, inplace = 1): 
                print(line.replace(original_name, new_name))
            return file
            
                            
        

In [72]:
name_to_replace= {"Speaker 1": "Chercheur", "Speaker 0": "Enquêtée"}

n = 0
for nom, new_name in name_to_replace.items():
    n +=1
    if n == 1:
        nom_fichier = pseudonymiser(nom, new_name, file = "../records/2026-02-05_grazia_borrini_in_block.txt", column= "speaker", replace_file=False)
    else:
        pseudonymiser(nom, new_name, file = nom_fichier, column= "pseudo_speaker", replace_file=True)

# Compter un ou plusieurs mots

Nous créons maintenant un compteur de mots (équivalent à la fonction "find" dans word). On utiliser pour cela des expressions régulières (Regex). Les regex permettent par exemple de retrouver tous les mots contenant une même racine (ou terminaisons). Dans l'exemple ci-dessous, nous allons rechercher tout les mots qui contiennent la racine "participat" : participation, participatif, participative, participatory, etc.

Nous allons faire la recherche sur deux types de fichier : un fichier texte (.txt) et un fichier csv.

## Trouver les occurences de l'adjectif "participative" dans l'entretien de Grazia Borrini

Nous commençons par explorer l'entretien de Grazia Borrini en utilisant le fichier texte obtenu après avoir regroupés les segments en bloc. Ce fichier s'appelle `2026-02-05_grazia_borrini_in_block.txt`.

Pour cela, j'ouvre le fichier et je "mets" son contenu dans l'objet "texte". 

In [73]:
with open("../records/2026-02-05_grazia_borrini_in_block.txt","r") as f:
    texte = f.read()

len(texte)

3834932

Une fois l'objet `texte` créé, on peut effectuer différents décomptes :

* Si je veux connaître le nombre de caractères, j'exécute la commande `len(texte)` (len pour length).
* Si je veux connaître le nombre de mots/chaînes de caractères : `len(texte.split())`. La fonction `.split()` découpe le texte à chaque fois qu'elle trouve une espace.
* Si je veux connaître le nombre d'occurrence d'un mot quelconque : `texte.count("le mot que je cherche")`.

Essayez ces différentes commandes dans la cellule ci-dessous


In [74]:
texte.count("participative")

170

Bien sûr, il est possible d'inscrire les résultats dans des objets.

In [75]:
nb_caractere = len(texte)
nb_string = len(texte.split())
occ_participat = texte.count("participative")

print("Nombre de caractères ",nb_caractere) # affiche le nombre de chaînes de caractère
print("Longueur du texte ", nb_string) # affiche le nombre de chaînes de caractère
print("Occurence du terme 'participative' ", occ_participat) # compte le nombre de fois que participative apparaît

Nombre de caractères  3834932
Longueur du texte  662412
Occurence du terme 'participative'  170


Combien de fois le terme "participative" apparaît dans l'entretien de Grazia Borrini ? Quel est le problème avec cette clé de recherche ?

C'est en ce sens que les regex sont utiles, que ce soit dans le cadre des traitements que nous faisons, mais aussi dans tous les cas où vous faites une recherche sur word, le web, etc. Elles permettent de préciser la recherche en l'élargissant ou en la spécifiant. Par exemple `texte.count()` compte les formes singulières et plurielles. Mais parfois, il peut intéressant de compter le nombre de fois qu'un mot est mis au singulier ou mis au pluriel.

```
Il existe de nombreuses ressources sur le web pour comprendre le fonctionnement des regex et le rôle des différentes "balises". Vous pouvez en particulier consulter le tutoriel sur le site W3school : [https://www.w3schools.com/python/python_regex.asp](https://www.w3schools.com/python/python_regex.asp)```


In [76]:
test = "Il a une fois participative dans sciences participatives."

test.count("participative")

2

La fonction ci-dessous est similaire à la fonction test.count(mot_a_chercher)

In [77]:
re.findall(r"participative", test)

['participative', 'participative']

* Si on ne cherche que les formes au singulier:

In [78]:
re.findall(r"participative\b", test) # le "\b" indique que l'on recherche toutes les chaines de caractères finissant par "participative", c'est-à-dire le mot "participative" suivi d'un espace ou d'un signe de ponctuation

['participative']

* Si on ne cherche que les formes au pluriel:

In [79]:
re.findall(r"participatives", test)



['participatives']

**OU**

In [80]:
re.findall(r"participatives\b", test)

['participatives']

In [81]:
re.findall(r"participative\w\b", test)

['participatives']

* Si on cherche les deux formes, mais que l'on souhaite les distinguer

In [82]:
re.findall(r"participatives?", test) # Le point d'interrogation signifie qu'on cherche toutes les occurrence de participative contenant un "s" ou aucun.

['participative', 'participatives']

**OU**

In [83]:
re.findall(r"participatives*\b", test) # L'astérisque signifie qu'on cherche toutes les occurrence de participative contenant aucun "s" ou plus

['participative', 'participatives']

**OU**

In [84]:
re.findall(r"participative.", test) #La balise ".\b" signifie qu'on cherche toutes les occurrence de participative suivi de n'importe quel caractère en bout de chaîne

['participative ', 'participatives']

Dans la cellule ci-dessous, on recherche fois toutes les fois qu'apparaît la racine "participat" quelque soit la terminaison (participation, participatif, participatory, etc.)

In [85]:
words_with_participat = re.findall(r"participat\w*", texte.lower()) 
print("Le nombre de mots contenant la racine 'participat' est : ", len(words_with_participat))
print("Voici la liste des différentes formes retrouvées : ", set(words_with_participat))

Le nombre de mots contenant la racine 'participat' est :  1166
Voici la liste des différentes formes retrouvées :  {'participation', 'participatives', 'participative', 'participatory'}


En fait, la fonction `re.findall()` "retourne" la liste de toutes les occurrences des termes commençant par "participat". Si on veut la liste des différentes formes sans les doublons, on écrit l'objet contenant la liste à l'intérieur de la fonction `set()`.

In [86]:
print("La liste des occurrences de 'participat' : ", words_with_participat[0:10])
print("Voici la liste des différentes formes retrouvées sans les doublons : ", set(words_with_participat))

La liste des occurrences de 'participat' :  ['participatives', 'participatives', 'participative', 'participation', 'participative', 'participation', 'participative', 'participation', 'participative', 'participation']
Voici la liste des différentes formes retrouvées sans les doublons :  {'participation', 'participatives', 'participative', 'participatory'}


### Exercice

Pour chaque terme contenu dans la liste `words_with_participat`, comptez le nombre d'occurrences dans les entretiens de Grazia Borrini, puis dans celui de Marie Roué.

['les sciences participatives',
 'les sciences participatives',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'appeler recherche participative',
 'idée de participation',
 'de pratique participative',
 'idée de participation',
 'pour la participation',
 'pour la participation',
 'pour la participation',
 'pour la participation',
 'pour la participation',
 'avec cette participation',
 'avec cette participation',
 'avec cette participation',
 'avec cett

* Trouver une regex pour identifier les expressions contenant la racine "participat" comme "science participative" ou "démocratie participative".

### Mettre en gras les mots recherchés

Maintenant que nous avons identifiés les termes commençant par "participat", il peut être intéressant des les "taguer" dans le texte pour les repérer plus facilement à la lecture des entretiens. Ici, on choisit de les signaler en les encadrant de deux astérisques, mais on peut aussi utiliser des balises html, xml, etc.

In [21]:
with open("../records/2026-02-05_grazia_borrini_in_block.txt","r") as f:
    texte = f.readlines()

for s in texte:
    if re.search(r'participat\w+', s.lower()):
        for w in set(re.findall(r'participat\w+', s.lower())):
            s = re.sub(w, f'**{w}**', s.lower())
        print(s)


speaker 1 : <seg_1>je suis arrivé en post-doctorat au muséum national d'historie naturelle l'année dernière, en septembre. c'est un projet porté par à la fois stéphanie duvaille et les écologues qui ont mis au point ce qu'on appelle les sciences **participatives** ou en anglais les citizen science.</seg_1><seg_2>je suis arrivé en post-doctorat au muséum national d'historie naturelle l'année dernière, en septembre. c'est un projet porté par à la fois stéphanie duvaille et les écologues qui ont mis au point ce qu'on appelle les sciences **participatives** ou en anglais les citizen science.</seg_2>

speaker 1 : <seg_6>? et puis stéphanie duval et son labo qui est plutôt... qu'on pourrait appeler recherche **participative**. là aussi, il y a beaucoup de termes très proches, mais qui mettent des méthodes différentes. et c'est dans ce cadre-là qu'elle m'a indiqué qu'à la fois vos travaux et peut-être l'environnement, la communauté dans laquelle vous étiez, ont nourri cette idée de **particip

Si on veut enregistrer le résultat dans un fichier .txt

In [22]:
with open("../records/2026-02-05_grazia_borrini_in_block.txt" , 'w', newline='') as txt:
    for s in texte: 
        txt.write(s)

La cellule ci-dessous permet de faire la même recherche dans des données présentées dataframe (tableur)

In [87]:
dict_n_word = {}
dict_word = {}
for n, seg in enumerate(gb.id):
    row = gb.iloc[n]
    n_word = len(re.findall(r'participat\w+', row["text"].lower()))
    word = [x for x in set(re.findall(r'participat\w+', row["text"].lower()))]
    if n_word > 0:
        
        dict_word[seg]= "|".join(word)
        dict_n_word[seg] = n_word

dict_word
gb["partipat"] = gb.id.map(dict_word.get)
gb["nb_partipat"] = gb.id.map(dict_n_word.get)
gb.loc[gb.nb_partipat>0]



,id,start,end,speaker,text,start_in_ms,end_in_ms,bloc_id,bloc_text,start_bloc,end_bloc,start_bloc_ms,end_bloc_ms,partipat,nb_partipat
1,seg_2,"00:00:09,299","00:00:19,930",Speaker 1,C'est un projet porté par à la fois Stéphanie ...,9.299,19.930,bloc_0,Je suis arrivé en post-doctorat au Muséum Nati...,"00:00:01,820","00:00:09,259",1.820,9.259,participatives,1.0
6,seg_7,"00:00:35,597","00:00:43,638",Speaker 1,Et puis Stéphanie Duval et son labo qui est pl...,35.597,43.638,bloc_4,? Et puis Stéphanie Duval et son labo qui est ...,"00:00:35,517","00:00:35,597",35.517,35.597,participative,1.0
8,seg_9,"00:00:53,782","00:01:14,258",Speaker 1,Et c'est dans ce cadre-là qu'elle m'a indiqué ...,53.782,74.258,bloc_4,? Et puis Stéphanie Duval et son labo qui est ...,"00:00:35,517","00:00:35,597",35.517,35.597,participation|participative,2.0
10,seg_11,"00:01:24,204","00:01:37,453",Speaker 1,"Dans ce cadre, j'aimerais qu'on revienne un pe...",84.204,97.453,bloc_4,? Et puis Stéphanie Duval et son labo qui est ...,"00:00:35,517","00:00:35,597",35.517,35.597,participation,1.0
14,seg_15,"00:01:56,727","00:02:08,678",Speaker 0,"La raison de l'intérêt pour la participation, ...",116.727,128.678,bloc_5,"Oui, très volontiers. Mais en général, j'aimer...","00:01:37,553","00:01:40,375",97.553,100.375,participation,1.0
54,seg_55,"00:06:24,104","00:06:32,788",Speaker 0,"Alors, bon... Vous aimeriez faire quelque chos...",384.104,392.788,bloc_19,"Non, je ne connais pas tellement ce travail. O...","00:06:10,978","00:06:13,079",370.978,373.079,participation,1.0
136,seg_137,"00:17:08,477","00:17:27,063",Speaker 0,"Et c'est sur ça que, moi je travaillais avec b...",1028.477,1047.063,bloc_21,J'étais en train de lire un article sur la rel...,"00:07:15,325","00:07:24,143",435.325,444.143,participatory,2.0
140,seg_141,"00:17:43,408","00:17:53,357",Speaker 0,"Ça veut dire, oui, la participation de la comm...",1063.408,1073.357,bloc_23,"par ces médecins qui étaient là, qui d'une cer...","00:17:31,437","00:17:43,368",1051.437,1063.368,participation,1.0
183,seg_184,"00:23:29,051","00:23:41,761",Speaker 0,"Et là, c'était justement là qu'on a commencé à...",1409.051,1421.761,bloc_25,Et ce n'est pas seulement les tertiaires ou le...,"00:17:54,715","00:17:59,058",1074.715,1079.058,participatives,1.0
184,seg_185,"00:23:41,801","00:24:07,136",Speaker 0,Car ces méthodes participatives donnaient la f...,1421.801,1447.136,bloc_25,Et ce n'est pas seulement les tertiaires ou le...,"00:17:54,715","00:17:59,058",1074.715,1079.058,participatives,1.0


Le problème de cette méthode est qu'il faut connaître a priori les termes que l'on souhaite rechercher. Nous allonrs maintenant voir comment faire ressortir les termes spécifiques de l'entretien.

# Identifier les termes "spécifiques" d'un entretien

## Le Term Frequency-Inverse Document Frequency (TF-IDF)

Le "Term Frequency-Inverse Document Frequency" (TF-IDF) est un indice statistique utilisé dans la recherche d'information et l'exploration de documents. Il permet de "mesurer" l'importance d'un terme dans un document comparativement à un ensemble de documents. Au sens du "TF-IDF", un mot est important dans un document lorsqu'il apparait plus fréquement que dans le reste du corpus. Par exemple, si dans un entretien l'expression "savoirs locaux" apparaît 10 fois alors que dans les autres il n'apparaît jamais, on peut supposer que c'est une expression "importante" pour la personne qui la cite.

Les éléments qui vont suivre permettent de calculer ce TF-iDF. Nous allons d'abord voir comment le calculer 'from scratch', afin de comprendre les différents élements qui entre dans le calcul de l'indice au cas où voudriez le faire dans excel par exemple. Puis, nous utiliserons des modules pythons qui permettent de le faire de façon plus "concise".

Le calcul du TF-IDF suppose au préalable de "nettoyer" les textes : le passer en petite casse, retirer les signes de ponctuation et les "stop words" (mots vides en français). Les "stop words" sont des mots tellement communs qu'ils n'apportent théoriquement aucune information sur le contenu du texte. Les articles (la, le, les, de, du, des, etc.), les auxilliaires (être, avoir) sont généralement considérés comme des stop words. 

Plusieurs modules sous Python proposent des listes de "stop words". Je vous propose d'utiliser les modules "Spacy" et "Nltk" conçues spécialement pour le traitement automatique des langues.

### Installer et charger Spacy

Spacy n'est pas encore installé. Pour cela nous ouvrons une cellule de code et écrivons `!pip install spacy`

In [89]:
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 102.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.9/835.9 kB 75.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 87.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 93.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 98.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21/21 [spacy]m20/21 [spacy]wn-it-py]

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


Une fois spacy installé, nous allons télécharger le modèle de langue dont on n'a besoin. Dans le cas des entretiens utilisés pour l'atelier, on prend le modèle français. Mais il existe d'autres langues : l'anglais bien sûr, mais aussi l'allemand, le chinois, le polonais, etc. 

Vous trouverez les modèles de langue existant ici  : [https://spacy.io/models](https://spacy.io/models)



In [91]:
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 100.0 MB/s  0:00:00eta 0:00:01

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


### Les stop words proposés par Spacy

Enfin, on peut importer le module spacy, charger le modèle et imprimer la liste des stop words

In [95]:
import spacy
nlp = spacy.load('fr_core_news_md')
print(f"Il y a {len(nlp.Defaults.stop_words)} termes dans la liste des stopwords de Spacy")
print(nlp.Defaults.stop_words)

Il y a 507 termes dans la liste des stopwords de Spacy
{'celui', 'tout', 'huit', 'devra', 'hi', 'mes', 'ha', 'ces', 'nos', 'quant', 'c’', 'stop', 'mêmes', 'aurait', 'nous', 'procedant', 'dix-sept', "s'", 'quatre', "quelqu'un", 'diverse', 'auraient', 'trois', 'aupres', 'etaient', 'serait', 'auxquelles', 'qui', 'quelconque', 'dix-neuf', 'quoi', 'ce', 'da', 'doit', 'tienne', 'fait', 'vais', 't’', 'comment', 'aussi', 'aie', 'là', 'ô', 'apres', 'anterieure', 'envers', 'elles-memes', 'merci', 'o', 'mon', 'elle-meme', 'cinq', 'afin', 'moindres', 'vingt', 'memes', 'seules', 'cinquantaine', 'dessus', 'lui-meme', 'ho', 'onzième', 'siennes', 'après', 'néanmoins', 'plusieurs', 'première', 'nombreuses', 'es', 'revoilà', 'jusqu', 'celle-là', 'relative', 'trente', 'cet', 'notre', 'ton', 'avais', 'semblable', 'comme', 'faisant', 'septième', 'ai', 'votres', 'suivante', 'ta', 'jusque', 'alors', 'chacun', 'facon', 'rendre', 'du', 'bat', 'celui-la', 'quelle', 'certains', 'egalement', 'anterieures', 'tienn

### Les stop words proposés par le module nltk

In [99]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [102]:
stopwords.words("french")
print(f"Il y a {len(stopwords.words("french"))} termes dans la liste des stopwords de Spacy")
print(stopwords.words("french"))

Il y a 157 termes dans la liste des stopwords de Spacy
['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 

### Nettoyage du texte

Nous allons "nettoyer" le texte de l'entretien de Grazia Borrini et le "tokenizer" (i.e. diviser en mot). Nous allons par ailleurs travailler au niveau des blocs pour voir quels sont les thèmes qui émergent au fil de l'entretien ou ceux qui reviennent. Par défaut, aucun stopword n'est retiré.





In [196]:
def cleaner(text, lower_case = True, remove_stopword=None):
    if lower_case == True:
        text = text.lower()
    else:
        pass
    text = re.sub(r"http\S+", "", text) # remove urls
    text = re.sub("’", "'",text) #replace "’" by "'"
    text = re.sub("'", "' ",text) #replace "’" by "'"
    text = " ".join(re.sub("[^a-zA-Zàâäéèêëïîôöùûüÿçß\\-]+", " ", text).lower().split()) #remove hashtag, arobase, HTML character
    text = re.sub(r"\b[a-zA-Z].\b", "", text)
    if remove_stopword == None:
        split_text = text.split() # 
    else:
        split_text = [x for x in text.split() if x not in remove_stopword]
        text = ' '.join(split_text)
    
    return text, split_text

La fonction `cleaner()` prend en entrée un texte (un string en langage informatique). L'argument "remove_stopword" prend une liste de termes que l'on souhaite retirer pour l'analyse. Elle retourne ensuite deux objets : 
* la phrase "nettoyée", c'est-à-dire sans ponctuation, sans caractères spéciaux et en petite casse (on peut préciser ̀`lower_case = False` si on veut garder les majuscules)
* la phrase tokenizée

**Essayons-là!**

In [198]:
phrase = "Le chat mange une souris. La souris est mangée par un chat. La souris que le chat a mangée! #souris #chat. C'est le chat qui mange la souris."

cleaned_phrase, token_phrase = cleaner(phrase)
print("La phrase nettoyée : ", cleaned_phrase)
print("La phrase tokenizée : ", token_phrase)

La phrase nettoyée :   chat mange une souris  souris est mangée par  chat  souris que  chat mangée souris chat est  chat qui mange  souris
La phrase tokenizée :  ['chat', 'mange', 'une', 'souris', 'souris', 'est', 'mangée', 'par', 'chat', 'souris', 'que', 'chat', 'mangée', 'souris', 'chat', 'est', 'chat', 'qui', 'mange', 'souris']


La même phrase avec les "stop_words en moins":

In [199]:
fr_stopwords = stopwords.words("french") # on met les stop words de nltk dans une liste
cleaned_phrase, token_phrase = cleaner(phrase, remove_stopword=fr_stopwords)
print("La phrase nettoyée : ", cleaned_phrase)
print("La phrase tokenizée : ", token_phrase)

La phrase nettoyée :  chat mange souris souris mangée chat souris chat mangée souris chat chat mange souris
La phrase tokenizée :  ['chat', 'mange', 'souris', 'souris', 'mangée', 'chat', 'souris', 'chat', 'mangée', 'souris', 'chat', 'chat', 'mange', 'souris']


### Exercice

Utiliser la même fonction `cleaner` en utilisant cette fois la liste des stopwords proposée par Spacy

In [200]:
# Ecrire le code dans cette cellule


Nous allons maintenant répéter la fonction de nettoyage sur chacun des segments de l'entretien de Grazia Borrinin en utilisant le dataframe `gb`

In [205]:
fr_stopwords = list(nlp.Defaults.stop_words)#stopwords.words("french")

dict_token = {} #dictionnaire pour "conserver" la liste des tokens de chaque segment
dict_text = {} #dictionnaire pour "conserver" les segments nettoyés
for n, seg in enumerate(gb.id): # pour chaque segment dans la colonne gb.id
    row = gb.iloc[n] #on extrait lla ligne
    text = row['text']#on extrait le texte
    clean_text, split_text = cleaner(text, lower_case = True, remove_stopword=fr_stopwords)
    dict_token[seg]="|".join(split_text)
    dict_text[seg]= clean_text

gb["cleaned_text"] = gb.id.map(dict_text.get)
gb["tokens"] = gb.id.map(dict_token.get)

gb["tokens"]= gb.tokens.str.split("|")
gb_explode = gb[["id","bloc_id","speaker","tokens"]].explode("tokens")
gb_explode.tokens.value_counts()

tokens
faire         87
beaucoup      81
vraiment      65
choses        55
chose         51
              ..
récolte        1
données        1
labo           1
implémenté     1
blablabla      1
Name: count, Length: 1791, dtype: int64

In [202]:
gb_explode.loc[gb_explode.tokens=="c"]
gb.loc[gb.id == "seg_2"]

,id,start,end,speaker,text,start_in_ms,end_in_ms,bloc_id,bloc_text,start_bloc,end_bloc,start_bloc_ms,end_bloc_ms,partipat,nb_partipat,cleaned_text,tokens
1,seg_2,"00:00:09,299","00:00:19,930",Speaker 1,C'est un projet porté par à la fois Stéphanie ...,9.299,19.93,bloc_0,Je suis arrivé en post-doctorat au Muséum Nati...,"00:00:01,820","00:00:09,259",1.82,9.259,participatives,1.0,projet porté fois stéphanie duvaille écologues...,"[projet, porté, fois, stéphanie, duvaille, éco..."


In [207]:
import numpy as np

In [209]:
token_freq = gb_explode.groupby(["speaker","tokens"]).agg(term_freq=("speaker","size")).reset_index()
size_seg = gb_explode.groupby(["speaker"]).agg(nb_tokens=("tokens","size")).reset_index()
nb_of_seg = gb_explode.speaker.nunique()
nb_seg_with_freq = gb_explode[["speaker","tokens"]].drop_duplicates().groupby("tokens").agg(nb_doc=("speaker","size")).reset_index()

df_tf = token_freq.merge(size_seg, on = ["speaker"], how = "left").merge(nb_seg_with_freq, on = ["tokens"], how = "left")
df_tf["tf"]= df_tf.term_freq/df_tf.nb_tokens
df_tf["idf"]= np.log((nb_of_seg+1)/df_tf.nb_doc)+1
df_tf["tfidf"]= df_tf.tf*df_tf.idf
df_tf.loc[df_tf.speaker == "Speaker 1"].sort_values(by="tfidf", ascending = False)

,speaker,tokens,term_freq,nb_tokens,nb_doc,tf,idf,tfidf
1622,Speaker 1,,13,573,2,0.022688,1.405465,0.031887
1740,Speaker 1,faire,11,573,2,0.019197,1.405465,0.026981
1649,Speaker 1,avez,9,573,2,0.015707,1.405465,0.022075
1698,Speaker 1,coup,6,573,1,0.010471,2.098612,0.021975
1792,Speaker 1,justement,8,573,2,0.013962,1.405465,0.019623
...,...,...,...,...,...,...,...,...
1931,Speaker 1,science,1,573,2,0.001745,1.405465,0.002453
1933,Speaker 1,scientifiques,1,573,2,0.001745,1.405465,0.002453
1912,Speaker 1,revenir,1,573,2,0.001745,1.405465,0.002453
1633,Speaker 1,année,1,573,2,0.001745,1.405465,0.002453


In [210]:
def compute_tfidf(data, group_by):
    token_freq = data.groupby([group_by,"tokens"]).agg(term_freq=(group_by,"size")).reset_index()
    size_seg = data.groupby([group_by]).agg(nb_tokens=("tokens","size")).reset_index()
    nb_of_seg = data.speaker.nunique()
    nb_seg_with_freq = data[[group_by,"tokens"]].drop_duplicates().groupby("tokens").agg(nb_doc=(group_by,"size")).reset_index()
    
    df_tf = token_freq.merge(size_seg, on = [group_by], how = "left").merge(nb_seg_with_freq, on = ["tokens"], how = "left")
    df_tf["tf"]= df_tf.term_freq/df_tf.nb_tokens
    df_tf["idf"]= np.log((nb_of_seg+1)/df_tf.nb_doc)+1
    df_tf["tfidf"]= df_tf.tf*df_tf.idf
    return df_tf

In [211]:
gb_explode

,id,bloc_id,speaker,tokens
0,seg_1,bloc_0,Speaker 1,arrivé
0,seg_1,bloc_0,Speaker 1,post-doctorat
0,seg_1,bloc_0,Speaker 1,muséum
0,seg_1,bloc_0,Speaker 1,national
0,seg_1,bloc_0,Speaker 1,historie
...,...,...,...,...
930,seg_931,bloc_109,Speaker 0,exister
931,seg_932,bloc_109,Speaker 0,fiche
932,seg_933,bloc_110,Speaker 1,bon
932,seg_933,bloc_110,Speaker 1,essayer


In [214]:
df = compute_tfidf(data=gb_explode, group_by="bloc_id")
#dict_tfidf
df["id_bloc"] = df.bloc_id.str.replace("bloc_","").astype(int)
df = df.merge(gb_explode[["bloc_id", "speaker"]].drop_duplicates(), on = ["bloc_id"], how ="left")
df = df.loc[df.speaker == "Speaker 0"].sort_values("id_bloc", ascending=True)

for bloc in df.id_bloc.unique():
    df_bloc = df.loc[df.id_bloc == bloc].sort_values("tfidf", ascending=False)
    specific_term = [x for x in df_bloc.tokens.head(5)]
    print(f"Les 5 termes les plus spécifique du bloc {bloc} sont : ", specific_term)


Les 5 termes les plus spécifique du bloc 1 sont :  ['citizen', 'science']
Les 5 termes les plus spécifique du bloc 3 sont :  ['recherche', 'direction', 'peut-être', 'est-']
Les 5 termes les plus spécifique du bloc 5 sont :  ['allez', 'volontiers', 'choix', 'but', 'esprit']
Les 5 termes les plus spécifique du bloc 7 sont :  ['cadre-', 'ensemble', '']
Les 5 termes les plus spécifique du bloc 9 sont :  ['normaux']
Les 5 termes les plus spécifique du bloc 11 sont :  ['']
Les 5 termes les plus spécifique du bloc 13 sont :  ['voulez', 'aimeriez', 'vieille', 'passionnant', 'condominium']
Les 5 termes les plus spécifique du bloc 15 sont :  ['pâques', 'français']
Les 5 termes les plus spécifique du bloc 17 sont :  ['consortiums', 'oui']
Les 5 termes les plus spécifique du bloc 19 sont :  ['citoyens', 'efforts', 'aimeriez', 'superposent', 'changement']
Les 5 termes les plus spécifique du bloc 21 sont :  ['maison', 'physique', 'religion', 'lire', 'pouvons']
Les 5 termes les plus spécifique du blo

In [215]:
df.sort_values("id_bloc", ascending=True)

,bloc_id,tokens,term_freq,nb_tokens,nb_doc,tf,idf,tfidf,id_bloc,speaker
23,bloc_1,citizen,1,2,2,0.500000,1.405465,0.702733,1,Speaker 0
24,bloc_1,science,1,2,4,0.500000,0.712318,0.356159,1,Speaker 0
1363,bloc_3,est-,1,4,19,0.250000,-0.845827,-0.211457,3,Speaker 0
1362,bloc_3,direction,1,4,4,0.250000,0.712318,0.178079,3,Speaker 0
1364,bloc_3,peut-être,1,4,13,0.250000,-0.466337,-0.116584,3,Speaker 0
...,...,...,...,...,...,...,...,...,...,...
341,bloc_109,sérieuse,1,77,1,0.012987,2.098612,0.027255,109,Speaker 0
340,bloc_109,souffrance,1,77,2,0.012987,1.405465,0.018253,109,Speaker 0
339,bloc_109,sociales,1,77,2,0.012987,1.405465,0.018253,109,Speaker 0
338,bloc_109,sens,2,77,3,0.025974,1.000000,0.025974,109,Speaker 0


## TF-IDF avec scikit learn

In [216]:
# importer le vectoriseur TfidfVectorizer de Scikit-Learn.  
from sklearn.feature_extraction.text import TfidfVectorizer

In [217]:
# Chargement du modèle français de SpaCy
nlp = spacy.load('fr_core_news_md')  # ou 'en_core_web_sm' pour l'anglais
# Récupération et extension des stopwords par défaut de SpaCy
spacy_stopwords = set(nlp.Defaults.stop_words)



# Application de la lemmatisation et de la suppression des stopwords sur chaque document


In [218]:
# Fonction pour lemmatiser le texte et retirer les stopwords
def lemmatize_and_remove_stopwords(text, nlp, stopwords, lemmatiser = True):
    doc = nlp(text)
    if lemmatiser == True:
        cleaned_text = " ".join([token.lemma_ for token in doc if token.text.lower() not in stopwords and not token.is_punct])
    else:
        cleaned_text = " ".join([str(token) for token in doc if token.text.lower() not in stopwords and not token.is_punct])
    return cleaned_text

In [219]:
print(list_bloc_text[0]["bloc_text"])
print(lemmatize_and_remove_stopwords(list_bloc_text[0]["bloc_text"], nlp, spacy_stopwords))

Donc je vais vous présenter un peu, redire un peu pourquoi on se rencontre et puis on intègre pas l'entretien. Donc moi je suis Aymeric et j'ai été recruté en post-doctorat pour travailler sur la question des observatoires participatifs.
présenter redire rencontre intégrer entretien Aymeric être recruter post-doctorat travailler question observatoire participatif


In [220]:
tous_documents = []
for bloc in list_bloc_text:
    lemmatized_an_cleaned_text = lemmatize_and_remove_stopwords(bloc["bloc_text"], nlp, spacy_stopwords, lemmatiser = True)
    tous_documents.append(lemmatized_an_cleaned_text)

vectoriseur = TfidfVectorizer(max_df=.65, min_df=1, use_idf=True)
documents_transformes = vectoriseur.fit_transform(tous_documents)

In [221]:
documents_transformes_tableau = documents_transformes.toarray()
len(documents_transformes_tableau)

156

In [222]:
i = -1
for compteur, document in enumerate(documents_transformes_tableau):
    i+=1
    #print("#####\n",document)
    # construire un objet de la classe DataFrame
    tf_idf_tuples = list(zip(vectoriseur.get_feature_names_out(), document))
    df0 = pd.DataFrame.from_records(tf_idf_tuples, columns=['terme', 'pointage']).sort_values(by='pointage', ascending=False).reset_index(drop=True)
    df0 = df0.loc[df0.pointage > 0]
    df0['bloc_id'] = str(compteur)
    if i == 0:
        df = df0.copy()
    else:
        df = pd.concat([df, df0])

for bloc in df.bloc_id.unique():
    df_bloc = df.loc[df.bloc_id == bloc].sort_values("pointage", ascending=False)
    specific_term = [x for x in df_bloc.terme.head(5)]
    print(f"Les 5 termes les plus spécifique du bloc {bloc} sont : ", specific_term)


Les 5 termes les plus spécifique du bloc 0 sont :  ['recruter', 'aymeric', 'observatoire', 'post', 'présenter']
Les 5 termes les plus spécifique du bloc 1 sont :  ['formation', 'doctorat', 'être']
Les 5 termes les plus spécifique du bloc 2 sont :  ['sociologie', 'former', 'sociologu', 'thèse', 'accord']
Les 5 termes les plus spécifique du bloc 3 sont :  ['sociologie', 'environnement']
Les 5 termes les plus spécifique du bloc 4 sont :  ['riverain', 'environnemental', 'engager', 'conflit', 'travailler']
Les 5 termes les plus spécifique du bloc 5 sont :  ['riverain']
Les 5 termes les plus spécifique du bloc 6 sont :  ['riverain', 'pollution', 'engager', 'problème']
Les 5 termes les plus spécifique du bloc 7 sont :  ['accord']
Les 5 termes les plus spécifique du bloc 8 sont :  ['autour', 'hypersensibilité', 'marseille', 'controversée', 'chimique']
Les 5 termes les plus spécifique du bloc 9 sont :  ['ok']
Les 5 termes les plus spécifique du bloc 10 sont :  ['pratique', 'regard', 'proposer',